# Curso Análisis de Redes

## Cátedra Teoría de las Telecomunicaciones - Universidad ORT Uruguay



# Control de congestión en redes (3a. parte)

In [1]:
#using Pkg;Pkg.instantiate(); Pkg.update()

## El problema de asignación de recursos

El problema de control de congestión en redes es un caso particular del problema de *asignación de recursos*.

* Existe un bien *escaso* a repartir: el ancho de banda de los enlaces.

* Las conexiones solicitan recursos a lo largo de su ruta.

* Los enlaces imponen *restricciones de capacidad*.

> **Problema:**
>
> ¿Cuánto ancho de banda se debe asignar a cada conexión respetando las restricciones de los enlaces?


### Ejemplo

En la siguiente red con $3$ conexiones compartiendo $2$ enlaces:

* ¿Cuáles son las asignaciones factibles?
* ¿Cuál es la asignación más lógica?

![Parking lot](images/parking_lot.png)

### Restricciones de capacidad

Las *restricciones de capacidad* de la red anterior son:

$$x_1 + x_2 \leqslant C_1,$$

$$x_1 + x_3 \leqslant C_2.$$

Las asignaciones factibles son el conjunto:

$$\mathcal{C} = \{(x_i): x_i\geqslant 0, x_1+x_2\leqslant C_1, x_1+x_3\leqslant C_2\}$$

denominado **región de capacidad de la red**.

### Restricciones de capacidad: diagrama

Para el caso del ejemplo, la región $\mathcal{C}$ tiene la siguiente forma:
![Parking Lot constraints](images/parking_lot_constraints.png)

### Matriz de ruteo

Una forma concisa de expresar la región de capacidad es considerar la matriz:

$$R = (R_{lr}) \quad R_{lr}= \begin{cases} 1 \text{ si la ruta $r$ pasa por el enlace $l$,}\\
                                           0 \text{ en otro caso.}\end{cases}$$ 
                                           
Si $C=(c_l)$ es un vector columna con las capacidades de los enlaces, entonces la región de capacidad es:

$$\mathcal{C} = \{x: x\geqslant 0, Rx\leqslant C\}$$

donde las desigualdades son coordenada a coordenada.

**Observación:** al ser un conjunto de desigualdades lineales, el resultado siempre es un politopo en la región $x_i\geqslant 0$.

### Matriz de ruteo: ejemplo.

En el ejemplo anterior:

$$R = \begin{pmatrix} 1 & 1 & 0 \\1 & 0 & 1\end{pmatrix} \quad x=\begin{pmatrix}x_1\\x_2\\x_3\end{pmatrix} \quad C=\begin{pmatrix}C_1\\C_2\end{pmatrix}.$$

## Asignaciones eficientes

La descripción anterior nos muestra cuáles asignaciones son *factibles*. Pero la asignación $x=(0,\ldots,0)$ siempre es factible pero no es deseable. Surge entonces la pregunta de qué asignaciones son *eficientes*.

> **Definición:** (Pareto-eficiencia)
>
> Una asignación $x\in \mathcal{C}$ es Pareto-eficiente si y solo si no se puede incrementar la asignación de $x_i$ sin empeorar algún $x_j$ con $j\neq i$.
>
> Matemáticamente, $x$ es Pareto eficiente si para toda $x'\in \mathcal{C}$ con $x'_i>x_i$ existe $j$ tal que $x'_j<x_j$.

### Ejemplo

Para el ejemplo anterior, la región Pareto-eficiente es la indicada en color azul:

<img src="images/parking_lot_pareto.png" width=700px />

## Asignación de máximo throughput

Del conjunto de asignaciones, puede tener sentido elegir aquella que *transporta más información* por la red. Esta es la asignación de máximo throughput, y que puede definirse mediante el siguiente problema:

> **Problema** (max-throughput):
>
> $$\max \sum_i x_i$$
>
> sujeto a:
>
> $$Rx\leqslant C,$$
> $$x\geqslant 0.$$

Es decir, maximizar la asignación total sujeto a las restricciones de capacidad. Denotaremos por $x^+$ la solución de este problema.

### Solución para la red de ejemplo.

El problema anterior es un *problema de optimización lineal* o *programación lineal*. Se trata de optimizar una función objetivo lineal sujeto a restricciones de desigualdad lineales (semiespacios).

> **Propiedad**: siempre existe un vértice del politopo $\mathcal{C}$ que es solución del problema.

Existen múltiples algoritmos para encontrar el óptimo (ej: el [método simplex](https://en.wikipedia.org/wiki/Simplex_algorithm) que recorre los vértices ordenadamente). En Julia lo implementamos con la bilbioteca `JuMP` que permite resolver problemas de optimización, y utilizando el solver gratuito `Ipopt`.

In [3]:
using JuMP, Ipopt

C1=100.0
C2=80.0

## Defino el problema de optimización de maximo throughput
model = Model(
           optimizer_with_attributes(
               Ipopt.Optimizer, "tol" => 1e-12, "print_level" => 0
    )
)

@variable(model,x[1:3]>=0)

con1 = @constraint(model, x[1]+x[2]<=C1)
con2 = @constraint(model, x[1]+x[3]<=C2)

@objective(model,Max,sum(x))

#Resuelvo
optimize!(model)

#Imprimo la salida
println("Asignación:")
println("x1=$(value(x[1]))")
println("x2=$(value(x[2]))")
println("x3=$(value(x[3]))")

Asignación:
x1=0.0
x2=100.00000100999975
x3=80.00000080999975


### Diagrama:

Volviendo al diagrama de capacidad:

<img src="images/parking_lot_max_throughput.png" width=700px />

### Eficiencia vs. justicia

La solución de máximo throughput es entonces:

$$x_1^+ = 0, \quad x_2^+ = C_1, \quad x_3^+=C_2,$$

Claramente maximiza el throughput (nunca puede ser más que $C_1+C_2$ y este alcanza dicha cota) pero penaliza excesivamente la ruta larga!

**Observación:** este resultado es general. Al querer maximizar el throughput, la asignación le quita recursos a las rutas que usan más de un enlace (porque dicha capacidad asignada cuenta una vez sola) y le da preferencia a las rutas cortas.

**Conclusión:** no siempre lo maś eficiente es lo más justo.

## Max-min fairness

Se busca entonces una asignación que reparta los recursos de manera "justa". Tomando ideas de la economía, se define la asignación *max-min fair* o de *justicia max-min* como sigue:

> **Asignación max-min**:
>
> La asignación $x^*\in \mathcal{C}$ es *max-min fair* si verifica la siguiente propiedad: para toda otra asignación factible, no se puede incrementar la asignación de una ruta sin disminuir alguna *que ya tenía menos*.
>
> Matemáticamente, si $x'\in\mathcal{C}$ y $x'_i >x^*_i$ entonces existe $j$ tal que $x^*_j\leqslant x^*_i$ y $x'_j<x^*_j$.

### Propiedades

La asignación max-min verifica las siguientes propiedades:

1. Es una asignación Pareto-eficiente.

2. Para toda otra asignación $x\in \mathcal{C}$ se tiene que:
$$\min_i x_i^* \geqslant \min_i x_i$$

Por lo tanto, el que se lleva menos, se lleva lo más posible.

### Algoritmo de water-filling

El cálculo de la asignación max-min se hace mediante el *algoritmo de water-filling* que se describe a continuación:

> **Algoritmo de water-filling:**
>
> 1. Se comienza con $x=(0,\ldots,0)$.
> 2. Se incrementa todas las fuentes por igual hasta saturar un enlace.
> 3. Todas las fuentes que comparten dicho enlace se fijan.
> 4. Se vuelve al paso 2 incrementando solo las fuentes que no se fijaron.
> 5. El algoritmo termina cuando todas las fuentes están fijas.

### Ejemplo

Para la red de ejemplo:

1. Comenzamos con $x=(0,0,0)$.
2. Incrementamos parejo hasta llegar a $x=(40,40,40)$. En ese momento satura el enlace $2$ ($x_1+x_3=80$).
3. Fijamos las fuentes $x_1^*=40$ y $x_3^*=40$ y queda libre la $2$.
4. Incrementamos la fuente $2$ hasta que $x_2^* = 60$, saturando el enlace $1$. Todas las fuentes están fijas.

Resultado:

$$x_1^* = 40, \quad x_2^*=60, \quad x_3^*=40.$$


### Diagrama:

<img src="images/parking_lot_max_min.png" width=600px />

In [5]:
model = Model(
           optimizer_with_attributes(
               Ipopt.Optimizer, "tol" => 1e-12, "print_level" => 0
    )
)

@variable(model,x[1:3]>=0)
@variable(model, z>=0)

con1 = @constraint(model, x[1]+x[2]<=C1)
con2 = @constraint(model, x[1]+x[3]<=C2)

@objective(model,Max,sum(x))

optimize!(model)

println("Asignación:")
println("x1=$(value(x[1]))")
println("x2=$(value(x[2]))")
println("x3=$(value(x[3]))")

Asignación:
x1=0.0
x2=100.00000100999975
x3=80.00000080999975


### Relación entre max-min y max-throughput

Las asignaciones obtenidas fueron entonces:

* $x^+ = (0,100,80)$, throughput total $\sum_i x_i^+ = 180$.

* $x^* = (40,60,40)$, throughput total $\sum_i x_i^* = 140$.

es decir, sacrificamos throughput neto para "emparejar" la asignación. Observar que el mínimo sube de $0$ a $40$.



## Asignación TCP/Reno

Luego de analizadas teóricamente las posibles asignaciones, nos gustaría entender cuál es la asignación de recursos que logra TCP/Reno, con su mecanismo de *control distribuido* donde cada fuente regula su propia velocidad en feedback.

**Hipótesis:**

 * Cada fuente TCP opera en feedback con la probabilidad de pérdida siguiendo la fórmula de Mathis:
 
 $$ x_r = \frac{1}{RTT} \sqrt{\frac{1.5}{p_r}}.$$
 
 * Despreciamos los retardos de cola, por lo que el $RTT$ de la conexión es la suma de los $RTT$ de los enlaces que atraviesa.

### Modelo de enlace.

Como vimos antes, en el análisis de un cuello de botella, el enlace opera sin pérdidas hasta llegar a su capacidad, y luego inmediatamente la probabilidad de pérdida se vuelve positiva, regulando a las fuentes.

**Modelo:**

Modelamos un enlace como una no-linealidad que opera en uno de los siguientes estados:

 * *No saturado:* 
$$\sum_{r\in l} x_r < c_l \quad \text{y} \quad p_l=0.$$
         
 * *Saturado:*
$$\sum_{r\in l} x_r = c_l \quad \text{y} \quad p_l>0.$$


**Observación:** es similar al modelo del diodo ideal, donde o bien $v=0$ e $i>0$ o $i=0$ y $v>0$. Notar que en ambos casos siempre tenemos una ecuación.

### Pérdidas en una ruta

Finalmente, tenemos la siguiente aproximación. Supongamos que una ruta $r$ atraviesa los enlaces $l_1,l_2,\ldots,l_k$ en orden. La probabilidad de pérdida de la ruta $r$ es entonces:

$$p_r = p_{l_1} + (1-p_{l_1})p_{l_2} + (1-p_{l_1})(1-p_{l_2})p_{l_3} + \ldots + (1-p_{l_1})\cdots(1-p_{l_{k-1}})p_{l_k}.$$

Si las probabilidades de pérdida son pequeñas $p_{l}\ll 1$, podemos hacer la siguiente aproximación:

$$p_r = \sum_{l\in r} p_l = \sum_{r} R_{lr}p_l.$$

### Cálculo de la asignación TCP

Combinando todo lo anterior, debemos hallar, dada una red con $N$ rutas y $L$ enlaces debemos calcular:

 $$x_r, \; r=1,\ldots,N; \quad p_l, \; l=1,\ldots,L.$$

 En total $N+L$ incógnitas.

> **Algoritmo:** (Asignación TCP)
>
> 1. Se hace una hipótesis de saturación de enlaces.
> 2. Para los enlaces saturados, se utiliza $\sum_{r\in l} x_r = c_l$.
> 3. Para los enlaces no saturados, se utiliza $p_l=0$. En total $L$ ecuaciones de enlace.
> 4. Para cada fuente se utiliza la fórmula de Mathis:
> $$x_r = \frac{1}{RTT_r} \sqrt{\frac{1.5}{p_r}}$$
> con $p_r = \sum_{l\in r} p_l$. Total $N$ ecuaciones de fuente.
> 5. Se resuelve el sistema hallando la asignación $x^0$ de TCP.
> 6. Con la asignación hallada, se verifican todas las hipótesis de saturación, es decir:
>      * Si $p_l=0$, verifico $\sum_{r\in l} x^0_r < c_l$.
>      * Si $\sum_{r\in l} x^0_r = c_l$ verifico que $p_l>0$.

## Ejercicio

Considere la siguiente red, cuyas restricciones de capacidad se diagraman a continuación:

![Three links](images/three_links.png)

1. Discutir cuál es la región Pareto-eficiente.
2. Calcular cuál(es) asignaciones son de máximo throughput.
3. Calcular la asignación max-min mediante el algoritmo de water-filling.
4. Calcular la asignación de TCP/Reno. Puede suponer inicialmente que los enlaces saturados son los mismos que en max-min como hipótesis inicial (pero deberá verificarla).

### Restricciones de capacidad

<img src="images/three_link_constraints.png" width="700px" />